# Introducing automating the boring stuff in Excel with Python

#### Purpose: demo xlwings API and automation use cases for Excel

#### xlwings docs
https://docs.xlwings.org/en/stable/api.html


#### xlwings vs. openxl - see good Stack Overflow response below. 
https://stackoverflow.com/a/58331928

#### Notes
* xlwings requires Excel to be installed on your machine. So if you were doing server-side parsing or working from a non-Windows machine, openpyxl could perhaps be more attractive. However, xlwings tends to support higher-level table abstractions well (working with pandas dataframes) and xlwing will not overwrite your current workbook formatting. This is useful if you have any Excel-based reporting use cases that work from templates.
* Why else might I learn Python or xlwings? So you don't have to learn VBA ;)
    * VBA has many strengths including support for forms and automation for non-technical users (doesn't require installing anything)
    * Nice thing about Python is that it will glue well with other apps, file types (SQL, text/CSV), and the file system itself

In [ ]:
# Python gives you wings
import antigravity

In [1]:
import pandas as pd
import xlwings as xw
import requests
import os
import wget

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# source: National Transportation and Safety Board (NTSB) 
# Aviation Accident Database & Synopses
# https://www.ntsb.gov/_layouts/ntsb.aviation/index.aspx

url = r'http://app.ntsb.gov/aviationquery/Download.ashx?type=csv'
wget.download(url, 'AviationData.txt')

In [2]:
# inspect first 2 lines
with open('AviationData.txt', 'r') as f:
    print(f.readline())
    print(f.readline())

Event Id | Investigation Type | Accident Number | Event Date | Location | Country | Latitude | Longitude | Airport Code | Airport Name | Injury Severity | Aircraft Damage | Aircraft Category | Registration Number | Make | Model | Amateur Built | Number of Engines | Engine Type | FAR Description | Schedule | Purpose of Flight | Air Carrier | Total Fatal Injuries | Total Serious Injuries | Total Minor Injuries | Total Uninjured | Weather Condition | Broad Phase of Flight | Report Status | Publication Date | 

20191230X91852 | Accident | CEN20CA048 | 12/30/2019 | GRANBURY, TX | United States | 32.365556 | -97.645000 |  | N/A | Non-Fatal | Substantial | Airplane | N519RV | Vans | RV 10 | Yes |  |  | Part 91: General Aviation |  | Personal |  |  |  |  | 1 |  |  | Preliminary | 12/31/2019 | 



In [3]:
df = pd.read_csv('AviationData.txt', sep='|')

In [4]:
df.head()

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Airport Code,Airport Name,Injury Severity,Aircraft Damage,Aircraft Category,Registration Number,Make,Model,Amateur Built,Number of Engines,Engine Type,FAR Description,Schedule,Purpose of Flight,Air Carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Weather Condition,Broad Phase of Flight,Report Status,Publication Date,
0,20191230X91852,Accident,CEN20CA048,12/30/2019,"GRANBURY, TX",United States,32.365556,-97.645000,,N/A,Non-Fatal,Substantial,Airplane,N519RV,Vans,RV 10,Yes,,,Part 91: General Aviation,,Personal,,,,,1,,,Preliminary,12/31/2019,
1,20191219X84114,Accident,CEN20CA037,12/19/2019,"Beeville, TX",United States,28.367500,-97.796389,BEA,,Non-Fatal,Substantial,Helicopter,N695AP,Robinson,R22,No,1,,Part 91: General Aviation,,Instructional,,,,,1,VMC,,Preliminary,12/31/2019,
2,20191218X74518,Accident,WPR20CA048,12/18/2019,"Caldwell, ID",United States,43.643889,-116.636944,EUL,CALDWELL INDUSTRIAL,Non-Fatal,Substantial,Airplane,N1107C,Piper,PA22,No,1,,Part 91: General Aviation,,Personal,,,,,1,VMC,,Preliminary,12/18/2019,
3,20191219X84839,Accident,CEN20CA036,12/18/2019,"Starkville, MS",United States,,,,,Non-Fatal,Substantial,Airplane,N9566S,Champion,7ECA,No,,,Part 91: General Aviation,,,,,1,,,,,Preliminary,12/20/2019,
4,20191216X85434,Accident,WPR20CA045,12/16/2019,"Lancaster, CA",United States,,,WJF,,Unavailable,Substantial,,N4098N,Cessna,140,No,1,,Part 91: General Aviation,,,,,,,,,,Preliminary,12/26/2019,


In [5]:
cols_despaced = [c.replace(' ', '') for c in df.columns]
df.columns = cols_despaced

In [6]:
df['Location'].value_counts()

 ANCHORAGE, AK           372
 MIAMI, FL               186
 CHICAGO, IL             169
 ALBUQUERQUE, NM         165
 Anchorage, AK           158
                        ... 
 ANDROS IS, Bahamas        1
 CUERNAVACA, Mexico        1
 HUNT, TX                  1
 6 NM NORTH OF S, AZ       1
 STEAMBOAT SPNGS, CO       1
Name: Location, Length: 26587, dtype: int64

In [7]:
# data contains spaces, ex: ' United States '
us_bool = df['Country'].str.contains('United States')

df['State'] = df[us_bool].loc[:, 'Location'].str.split(', ', expand=True).iloc[:, 1]
df['State'] = df['State'].fillna('NA - Non US')

In [8]:
df.groupby(['Country', 'State']).agg('size')

Country                           State       
                                  NA - Non US      507
 Afghanistan                      NA - Non US       12
 Algeria                          NA - Non US        5
 American Samoa                   NA - Non US       10
 Angola                           NA - Non US       10
 Anguilla                         NA - Non US        1
 Antarctica                       NA - Non US        4
 Antigua And Barbuda              NA - Non US        2
 Argentina                        NA - Non US       89
 Aruba                            NA - Non US        2
 Australia                        NA - Non US      245
 Austria                          NA - Non US       52
 Bahamas                          NA - Non US      205
 Bahrain                          NA - Non US        4
 Bangladesh                       NA - Non US        4
 Barbados                         NA - Non US        3
 Belarus                          NA - Non US        4
 Belgium          

In [9]:
us_avstats = df[us_bool].copy()
us_avstats['State'] = us_avstats['State'].str.replace(' ', '')

In [10]:
states = list(us_avstats['State'].str.upper().unique())
states = sorted(states)

In [11]:
print(states)

['AK', 'AL', 'AO', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FN', 'GA', 'GM', 'GU', 'HI', 'HONOLULU', 'IA', 'ID', 'IL', 'IN', 'KAUAI', 'KS', 'KY', 'LA', 'MA', "MANU'A", 'MAUI', 'MD', 'ME', 'MH', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NA-NONUS', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'NYC', 'OAHU', 'OH', 'OK', 'ON', 'OR', 'PA', 'PO', 'PR', 'RI', 'SANJUANIS.', 'SC', 'SD', 'TN', 'TX', 'UN', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']


# First, try manual way... and explore the API / user commands

In [12]:
state_bool = us_avstats['State'] == 'IL'

In [13]:
state_df = us_avstats[state_bool]

In [14]:
# this is still pandas API (not xlwings)

state_df.to_excel('IL.xlsx', index=False)

In [15]:
# assign filename at save time
wb = xw.Book()


```python
# specify file name to open an existing file
wb = xw.Book('xw_demo.xlsx')
```

In [16]:
wb.sheets

Sheets([<Sheet [Book1]Sheet1>])

In [17]:
wb.sheets['Sheet1']

<Sheet [Book1]Sheet1>

#### Sidebar: Object-Oriented Programming (OOP)...
* If you've used VBA, you might notice that xlwings is also object-oriented and frequently uses dot operator commands. 
* OOP is a vast subject, but the gist for this context is that program objects/abstractions give us conceptual objects (a workbook, a worksheet, a cell) that hide the program implementation details. In this way, we mold the programming language to fit the problem rather than the other way around.
* Objects have data (attributes; think nouns) and procedures (methods/code/procedures; think verbs). 
* In Python, we often access data attributes via the . (dot) operator. Objects with methods are called using the .method_name() convention.
* For example, a Sheet object has a sheetname (data) and a .delete() method.

In [18]:
# add new sheet

wb.sheets.add('Sheet2')

<Sheet [Book1]Sheet2>

In [19]:
# write values to cells

wb.sheets['Sheet1'].range('A1').value = 1
wb.sheets['Sheet1'].range('B1').value = 2

In [20]:
# read values from cells

wb.sheets['Sheet1'].range('B1').value

2.0

In [21]:
# abbreviate sheet objects

sht = wb.sheets['Sheet1']

In [22]:
# write dataframe
# note: this is WAY easier than some other APIs that have you double-loop through each cell address by row and column

sht.range('A1').value = state_df

In [23]:
# write dataframe and omit index

wb.sheets['Sheet2'].range('A1').options(pd.DataFrame, index=False).value = state_df

In [24]:
# autofit columns or rows

wb.sheets['Sheet2'].autofit('columns')
# wb.sheets['Sheet2'].autofit('rows')

In [25]:
# read dataframe from range

read_df = wb.sheets['Sheet2'].range('A1').options(pd.DataFrame, expand='table').value
read_df.head()

,InvestigationType,AccidentNumber,EventDate,Location,Country,Latitude,Longitude,AirportCode,AirportName,InjurySeverity,AircraftDamage,AircraftCategory,RegistrationNumber,Make,Model,AmateurBuilt,NumberofEngines,EngineType,FARDescription,Schedule,PurposeofFlight,AirCarrier,TotalFatalInjuries,TotalSeriousInjuries,TotalMinorInjuries,TotalUninjured,WeatherCondition,BroadPhaseofFlight,ReportStatus,PublicationDate
EventId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20190830X60449,Accident,DCA19CA208,08/26/2019,"Chicago, IL",United States,41.9744,-87.9067,KORD,Chicago O'Hare International,Non-Fatal,,Airplane,N911DL,Mcdonnell Douglas,MD88,No,2,,Part 121: Air Carrier,SCHD,,"Delta Air Lines, Inc",,1,,151,VMC,,Preliminary,12/20/2019
20190919X51117,Accident,GAA19CA558,08/09/2019,"Chicago, IL",United States,41.7858,-87.7525,MDW,Chicago Midway Intl,Non-Fatal,Substantial,Airplane,N9837L,Cessna,172,No,1,,Part 91: General Aviation,,Personal,,,,,1,VMC,,Preliminary,09/24/2019
20190802X91659,Accident,GAA19CA481,08/02/2019,"Vandalia, IL",United States,38.9753,-89.1425,,N/A,Non-Fatal,Substantial,Helicopter,N1767L,Robinson,R66,No,,Turbo Shaft,Part 137: Agricultural,,Aerial Application,,,,,1,VMC,MANEUVERING,Factual,11/04/2019
20190806X52246,Accident,CEN19LA251,07/27/2019,"Chicago, IL",United States,41.7831,-87.745,MDW,Chicago Midway Intl,Non-Fatal,Substantial,Airplane,N27AW,Ryan,NAVION,No,1,Reciprocating,Part 91: General Aviation,,Personal,,,,,4,VMC,LANDING,Preliminary,08/26/2019
20190712X24035,Accident,CEN19LA220,07/11/2019,"Wheeling, IL",United States,42.1347,-87.8917,PWK,Chicago Executive Airport,Non-Fatal,Substantial,Airplane,N83BA,Icon,A-5,No,1,Reciprocating,Part 91: General Aviation,,Personal,,,,,1,VMC,APPROACH,Preliminary,08/01/2019


In [26]:
# retrieve sheetname as string 
# (else you get sheet object "repr"/formal string representation)

wb.sheets['Sheet2'].name

'Sheet2'

In [27]:
# rename sheets

wb.sheets['Sheet2'].name = 'IL'

In [28]:
# display repr

wb.sheets

Sheets([<Sheet [Book1]IL>, <Sheet [Book1]Sheet1>])

In [29]:
# add worksheet

wb.sheets.add('Sheet3')

<Sheet [Book1]Sheet3>

In [30]:
# use sheet.name to collect all sheetnames (ex: for looping later)

sheet_name_list = [s.name for s in wb.sheets]
sheet_name_list

['Sheet3', 'IL', 'Sheet1']

In [31]:
# save workbook with assigned file name
# default will be Book1/Book2/etc. if not specified

wb.save('xlwings_demo.xlsx')

In [32]:
# closing will close the workbook - not the application!
# this is important if you do many loops and open up new application instances
# these are separate processes that may consume much memory without any tear-down logic

wb.close()
# wb.quit()

# Next, generalize (functions!) 

In [33]:
# new workbook object

wb = xw.Book()

In [34]:
# recap of earlier section
# what can we generalize before we start looping?
# we want a super generic function for other projects that we can copy or import

wb.sheets.add('IL')
state_bool = us_avstats['State'] == 'IL'
state_df = us_avstats[state_bool]
wb.sheets['IL'].range('A1').options(pd.DataFrame, index=False).value = state_df
wb.sheets['IL'].autofit('columns')

In [35]:
wb.close()

In [36]:
# new workbook

wb = xw.Book()

In [37]:
def make_xl_subdf(wb: xw.Book, sheetname: str, consol_df: pd.DataFrame, subset_field: str) -> None:
    """Creates an Excel sheet comprising of a dataframe set based
    
    Example
    -------
    >>> add_state(wb=wb, sheetname='IL', consol_df=us_avstats, subset_field='State')
    
    """
    try:
        wb.sheets.add(sheetname)
    except ValueError:
        print(f"Sheet named '{sheetname}' may already be present in workbook")
        
    sub_bool = consol_df[subset_field] == sheetname
    sub_df = consol_df[sub_bool]
    wb.sheets[sheetname].range('A1').options(pd.DataFrame, index=False).value = sub_df
    wb.sheets[sheetname].autofit('columns')
    

In [38]:
make_xl_subdf(wb=wb, sheetname='IL', consol_df=us_avstats, subset_field='State')

In [39]:
wb.close()

In [40]:
print(states)

['AK', 'AL', 'AO', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FN', 'GA', 'GM', 'GU', 'HI', 'HONOLULU', 'IA', 'ID', 'IL', 'IN', 'KAUAI', 'KS', 'KY', 'LA', 'MA', "MANU'A", 'MAUI', 'MD', 'ME', 'MH', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NA-NONUS', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'NYC', 'OAHU', 'OH', 'OK', 'ON', 'OR', 'PA', 'PO', 'PR', 'RI', 'SANJUANIS.', 'SC', 'SD', 'TN', 'TX', 'UN', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']


In [41]:
wb = xw.Book()

In [42]:
for s in sorted(states, reverse=True):
    print(f"Adding {s}")
    make_xl_subdf(wb=wb, sheetname=s, consol_df=us_avstats, subset_field='State')

wb.sheets['Sheet1'].delete()

Adding WY
Adding WV
Adding WI
Adding WA
Adding VT
Adding VI
Adding VA
Adding UT
Adding UN
Adding TX
Adding TN
Adding SD
Adding SC
Adding SANJUANIS.
Adding RI
Adding PR
Adding PO
Adding PA
Adding OR
Adding ON
Adding OK
Adding OH
Adding OAHU
Adding NYC
Adding NY
Adding NV
Adding NM
Adding NJ
Adding NH
Adding NE
Adding ND
Adding NC
Adding NA-NONUS
Adding MT
Adding MS
Adding MP
Adding MO
Adding MN
Adding MI
Adding MH
Adding ME
Adding MD
Adding MAUI
Adding MANU'A
Adding MA
Adding LA
Adding KY
Adding KS
Adding KAUAI
Adding IN
Adding IL
Adding ID
Adding IA
Adding HONOLULU
Adding HI
Adding GU
Adding GM
Adding GA
Adding FN
Adding FL
Adding DE
Adding DC
Adding CT
Adding CO
Adding CA
Adding AZ
Adding AR
Adding AO
Adding AL
Adding AK


In [43]:
wb.save(r'./split_data/split_demo.xlsx')

# Reading dataframes

In [44]:
read_df = wb.sheets['IL'].range('A1').options(pd.DataFrame, expand='table').value
read_df.head()

,InvestigationType,AccidentNumber,EventDate,Location,Country,Latitude,Longitude,AirportCode,AirportName,InjurySeverity,AircraftDamage,AircraftCategory,RegistrationNumber,Make,Model,AmateurBuilt,NumberofEngines,EngineType,FARDescription,Schedule,PurposeofFlight,AirCarrier,TotalFatalInjuries,TotalSeriousInjuries,TotalMinorInjuries,TotalUninjured,WeatherCondition,BroadPhaseofFlight,ReportStatus,PublicationDate
EventId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20190830X60449,Accident,DCA19CA208,08/26/2019,"Chicago, IL",United States,41.9744,-87.9067,KORD,Chicago O'Hare International,Non-Fatal,,Airplane,N911DL,Mcdonnell Douglas,MD88,No,2,,Part 121: Air Carrier,SCHD,,"Delta Air Lines, Inc",,1,,151,VMC,,Preliminary,12/20/2019
20190919X51117,Accident,GAA19CA558,08/09/2019,"Chicago, IL",United States,41.7858,-87.7525,MDW,Chicago Midway Intl,Non-Fatal,Substantial,Airplane,N9837L,Cessna,172,No,1,,Part 91: General Aviation,,Personal,,,,,1,VMC,,Preliminary,09/24/2019
20190802X91659,Accident,GAA19CA481,08/02/2019,"Vandalia, IL",United States,38.9753,-89.1425,,N/A,Non-Fatal,Substantial,Helicopter,N1767L,Robinson,R66,No,,Turbo Shaft,Part 137: Agricultural,,Aerial Application,,,,,1,VMC,MANEUVERING,Factual,11/04/2019
20190806X52246,Accident,CEN19LA251,07/27/2019,"Chicago, IL",United States,41.7831,-87.745,MDW,Chicago Midway Intl,Non-Fatal,Substantial,Airplane,N27AW,Ryan,NAVION,No,1,Reciprocating,Part 91: General Aviation,,Personal,,,,,4,VMC,LANDING,Preliminary,08/26/2019
20190712X24035,Accident,CEN19LA220,07/11/2019,"Wheeling, IL",United States,42.1347,-87.8917,PWK,Chicago Executive Airport,Non-Fatal,Substantial,Airplane,N83BA,Icon,A-5,No,1,Reciprocating,Part 91: General Aviation,,Personal,,,,,1,VMC,APPROACH,Preliminary,08/01/2019


In [46]:
wb.close()

# Challenge -- do the reverse
* Say we have data sets split among multiple tabs (like this example) or among many XLSX files. 
How would you go about merging them?
* Another idea: execute a SQL from a string or file and load report data into a template

# Recap - we covered:
* opening new or existing workbooks
* handling the Sheet() object
    * adding and deleting sheets
    * retrieving sheetnames
* saving, closing, and quiting workbooks
* modifying cell values
* adding tabular data to cell ranges via dataframes
    * and formatting tweaks like ignoring the index and autofitting columns/rows
* sample use case: splitting a merged data set into multipe tabs